In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 2.33MB/s]


In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df=pd.read_csv('preprocessed_data.csv')#reading the data into DataFrame


In [4]:
df.head(4)#displaying top 4 four data


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [5]:
def preprocess(x):#removing the last character
  x=x[:-1]
  return x

In [6]:
df['source']=df['source'].apply(preprocess)#preprocessing on source data
df['target']=df['target'].apply(preprocess)#perprocessing on target data

In [7]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [8]:
df.shape#shape of the data


(2000, 2)

In [9]:
df=df[df['source'].apply(len)<170]#removing source datapoints having length greater than equal to 170
df=df[df['target'].apply(len)<200]#removing target datapoints having length greater than equal to 200

In [10]:
df.shape#shape of the data


(1990, 2)

In [151]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


Target:

In [152]:
target_tokenizer= Tokenizer()#tokenization on target
target_tokenizer.fit_on_texts(y_train)#fitting on ytrain
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3034


In [153]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [154]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding to maxlength

In [155]:
target_padded_docs_train.shape


(1970, 43)

In [156]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [157]:
target_padded_docs_test.shape


(20, 43)

Source:

In [158]:
source_tokenizer= Tokenizer()#tokenization on source
source_tokenizer.fit_on_texts(X_train)#fitting to X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

3702


In [159]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to sequence

In [160]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [161]:
source_padded_docs_train.shape

(1970, 43)

In [162]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding to maxlength

In [163]:
source_padded_docs_test.shape

(20, 43)

In [164]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional
target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))


In [165]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [166]:
#we are reshaping the dataset because the sparese_categorical_crossentropy requires data to be three dimensional
source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))


In [167]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


Model1:

In [171]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,512, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.LSTM(128, return_sequences=True)(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512, activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
3model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()                                                                                                                               

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 43)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 43, 512)           1895936   
_________________________________________________________________
lstm_8 (LSTM)                (None, 43, 128)           328192    
_________________________________________________________________
time_distributed_16 (TimeDis (None, 43, 512)           66048     
_________________________________________________________________
dropout_8 (Dropout)          (None, 43, 512)           0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 43, 3035)          1556955   
Total params: 3,847,131
Trainable params: 3,847,131
Non-trainable params: 0
_________________________________________________

In [172]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [173]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 59s 57s/step - loss: 7.7430 - accuracy: 0.3208 - val_loss: 2.7519 - val_accuracy: 0.7698
Epoch 2/50
2/2 [==============================] - 1s 305ms/step - loss: 3.4479 - accuracy: 0.6728 - val_loss: 2.5025 - val_accuracy: 0.7698
Epoch 3/50
2/2 [==============================] - 1s 316ms/step - loss: 3.5213 - accuracy: 0.6728 - val_loss: 1.9935 - val_accuracy: 0.7721
Epoch 4/50
2/2 [==============================] - 1s 314ms/step - loss: 2.7795 - accuracy: 0.6730 - val_loss: 1.5858 - val_accuracy: 0.7791
Epoch 5/50
2/2 [==============================] - 1s 313ms/step - loss: 2.2776 - accuracy: 0.6807 - val_loss: 1.5165 - val_accuracy: 0.7709
Epoch 6/50
2/2 [==============================] - 1s 313ms/step - loss: 2.2367 - accuracy: 0.6729 - val_loss: 1.5061 - val_accuracy: 0.7698
Epoch 7/50
2/2 [==============================] - 1s 317ms/step - loss: 2.2036 - accuracy: 0.6739 - val_loss: 1.4808 - val_accuracy: 0.7779
Epoch 8/50
2/2 [=====

In [174]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
#Beam Search
from math import log
from numpy import array
from numpy import argmax
import numpy as np
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	#print(sequences)
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - np.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		sequences = ordered[:k]
	return sequences
 

In [175]:
#prediction
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in x])
  return y
#calculating bleu score using beam search where K==3
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
Hey... Kick boxing on sunday is fully booked...
Actual Output: 
Hey, kick boxing on Sunday is fully booked.
Predicted Output for beam==3 : 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


hey on sunday is booked
hey on sunday is mobile
hey on sunday is on
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Oh... I juz checked n realize my lesson is at 440...Haha, tt means i'll cya den...
Actual Output: 
Oh. I just checked and realize my lesson is at 4:40. Haha, that means I'll see you then.
Predicted Output for beam==3 : 
oh i just i and my lesson is at point afternoon that means i you then
oh i just i and my lesson is at point afternoon that means i see then
oh i just i and my lesson is at point haha that means i you then
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Fetch us at 7pm
Actual Output: 
Fetch us at 7pm.
Predicted Output for beam==3 : 
corner us at
million us at
drink us at

In [176]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.38480691187655225
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.36163580184089766
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.3734191852349319
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


Model2:

In [186]:
input=tf.keras.layers.Input(shape=(43,))
embed=tf.keras.layers.Embedding(source_vocab_size,512, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))(embed)  
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(lstm1)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 43)]              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 43, 512)           1895936   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 43, 200)           490400    
_________________________________________________________________
time_distributed_19 (TimeDis (None, 43, 3035)          610035    
Total params: 2,996,371
Trainable params: 2,996,371
Non-trainable params: 0
_________________________________________________________________


In [187]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [188]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 4s 874ms/step - loss: 7.6971 - accuracy: 0.3259 - val_loss: 5.2202 - val_accuracy: 0.7698
Epoch 2/50
2/2 [==============================] - 1s 286ms/step - loss: 4.6866 - accuracy: 0.6728 - val_loss: 1.9247 - val_accuracy: 0.7698
Epoch 3/50
2/2 [==============================] - 1s 296ms/step - loss: 2.7107 - accuracy: 0.6728 - val_loss: 2.0169 - val_accuracy: 0.7698
Epoch 4/50
2/2 [==============================] - 1s 291ms/step - loss: 2.9326 - accuracy: 0.6728 - val_loss: 1.9303 - val_accuracy: 0.7698
Epoch 5/50
2/2 [==============================] - 1s 291ms/step - loss: 2.7293 - accuracy: 0.6737 - val_loss: 1.8979 - val_accuracy: 0.7698
Epoch 6/50
2/2 [==============================] - 1s 289ms/step - loss: 2.5013 - accuracy: 0.6717 - val_loss: 1.6204 - val_accuracy: 0.7709
Epoch 7/50
2/2 [==============================] - 1s 286ms/step - loss: 2.2420 - accuracy: 0.6735 - val_loss: 1.4918 - val_accuracy: 0.7756
Epoch 8/50
2/2 [====

In [189]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=10,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/10
2/2 [==============================] - 1s 313ms/step - loss: 0.7306 - accuracy: 0.8575 - val_loss: 0.9117 - val_accuracy: 0.8733
Epoch 2/10
2/2 [==============================] - 1s 292ms/step - loss: 0.7071 - accuracy: 0.8601 - val_loss: 0.9130 - val_accuracy: 0.8744
Epoch 3/10
2/2 [==============================] - 1s 296ms/step - loss: 0.6841 - accuracy: 0.8626 - val_loss: 0.9143 - val_accuracy: 0.8756
Epoch 4/10
2/2 [==============================] - 1s 286ms/step - loss: 0.6617 - accuracy: 0.8648 - val_loss: 0.9150 - val_accuracy: 0.8733
Epoch 5/10
2/2 [==============================] - 1s 290ms/step - loss: 0.6407 - accuracy: 0.8671 - val_loss: 0.9149 - val_accuracy: 0.8733
Epoch 6/10
2/2 [==============================] - 1s 294ms/step - loss: 0.6204 - accuracy: 0.8699 - val_loss: 0.9132 - val_accuracy: 0.8756
Epoch 7/10
2/2 [==============================] - 1s 293ms/step - loss: 0.6007 - accuracy: 0.8718 - val_loss: 0.9126 - val_accuracy: 0.8767
Epoch 8/10
2/2 [====

In [190]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
#Beam Search

from math import log
from numpy import array
from numpy import argmax
import numpy as np
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	#print(sequences)
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - np.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		sequences = ordered[:k]
	return sequences
 

In [191]:
#prediction
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in x])
  return y
#calculating bleu score using beam search where K==3
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
Hey... Kick boxing on sunday is fully booked...
Actual Output: 
Hey, kick boxing on Sunday is fully booked.
Predicted Output for beam==3 : 
hey on sunday is jo
hey on sunday is green
hey on sunday is 2nd
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
Oh... I juz checked n realize my lesson is at 440...Haha, tt means i'll cya den...
Actual Output: 
Oh. I just checked and realize my lesson is at 4:40. Haha, that means I'll see you then.
Predicted Output for beam==3 : 
oh i just i and my lesson is at 4 haha then i'll i'll you then
oh i just i and my lesson is at 4 haha then i'll i'll you you
oh i just i and my lesson is at 4 haha then i'll i'll you meet
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [192]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if i=='<PAD>':
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.3672876018410912
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.336078085901221
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.3697109749269815
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
